In [1]:
%pylab inline
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
lencoder = LabelEncoder()


Populating the interactive namespace from numpy and matplotlib


In [39]:
data = pd.read_csv('partial_clean_data.csv')

/Users/viviannchan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,52,111) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
data = data.drop(['pymnt_plan','addr_state','last_pymnt_d','last_pymnt_amnt', 'out_prncp', 'policy_code', 'zip_code', 'total_pymnt', 'total_rec_int','total_rec_late_fee',\
                  'total_rec_prncp', 'recoveries','emp_title','loan_status'],1)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348059 entries, 0 to 1348058
Columns: 108 entries, id to target
dtypes: float64(91), int64(3), object(14)
memory usage: 1.1+ GB


In [42]:
# make emp_length and grade into integers
data['emp_length'] = data['emp_length'].astype(str)
data['emp_length'] = data['emp_length'].apply(lambda x: x.replace("years","").replace("year","").replace("+","").replace("< 1","0").replace("nan","-1")).astype(int)

In [43]:
data['grade'] = data['grade'].astype(str)
data['grade'] = data['grade'].apply(lambda x: x.replace("A",'1').replace("B",'2').replace("C","3").replace("D","4").replace("E","5").replace("F","6").replace("G","7"))
data['grade'] = data['grade'].astype(int)

In [51]:
cols_object = data.loc[:, data.dtypes == object].columns
data[cols_object]

,sub_grade,home_ownership,verification_status,issue_d,earliest_cr_line,initial_list_status,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line
0,C4,MORTGAGE,Not Verified,Dec-2015,Aug-2003,w,NaN,Mar-2019,Individual,NaN,NaN
1,C1,MORTGAGE,Not Verified,Dec-2015,Dec-1999,w,NaN,Mar-2019,Individual,NaN,NaN
2,B4,MORTGAGE,Not Verified,Dec-2015,Aug-2000,w,NaN,Mar-2019,Joint App,Not Verified,NaN
3,F1,MORTGAGE,Source Verified,Dec-2015,Jun-1998,w,NaN,Mar-2018,Individual,NaN,NaN
4,C3,RENT,Source Verified,Dec-2015,Oct-1987,w,NaN,May-2017,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1348054,B2,OWN,Not Verified,Oct-2016,Jul-2004,f,NaN,Mar-2019,Individual,NaN,NaN
1348055,C3,MORTGAGE,Not Verified,Oct-2016,Mar-2002,f,NaN,Jul-2018,Individual,NaN,NaN
1348056,C4,MORTGAGE,Source Verified,Oct-2016,Jun-2011,f,NaN,Dec-2018,Individual,NaN,NaN
1348057,C1,RENT,Not Verified,Oct-2016,Aug-1997,f,NaN,Mar-2019,Individual,NaN,NaN


In [45]:
# non-ordinal object, using get_dummies
purpose_dummies = pd.get_dummies(data.purpose)
data = pd.concat([data, purpose_dummies], axis=1)
data = data.drop('purpose',1)

In [38]:
# application_dummies = pd.get_dummies(data.application_type)
# data = pd.concat([data, application_dummies], axis=1)
# data = data.drop('application_type',1)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348059 entries, 0 to 1348058
Columns: 121 entries, id to wedding
dtypes: float64(91), int64(5), object(11), uint8(14)
memory usage: 1.1+ GB


In [47]:
data.verification_status_joint.value_counts()

Not Verified       14872
Source Verified     6241
Verified            4482
Name: verification_status_joint, dtype: int64

In [50]:
data.application_type.value_counts()

Individual    1322259
Joint App       25800
Name: application_type, dtype: int64

In [55]:
data[data.application_type=='Individual']

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,68407277,3600.0,3600.0,36,13.99,123.03,3,C4,10,MORTGAGE,...,0,0,0,0,0,0,0,0,0,0
1,68355089,24700.0,24700.0,36,11.99,820.28,3,C1,10,MORTGAGE,...,0,0,0,0,0,0,0,1,0,0
3,68476807,10400.0,10400.0,60,22.45,289.91,6,F1,3,MORTGAGE,...,0,0,1,0,0,0,0,0,0,0
4,68426831,11950.0,11950.0,36,13.44,405.18,3,C3,4,RENT,...,0,0,0,0,0,0,0,0,0,0
5,68476668,20000.0,20000.0,36,9.17,637.58,2,B2,10,MORTGAGE,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348054,89905081,18000.0,18000.0,60,9.49,377.95,2,B2,5,OWN,...,1,0,0,0,0,0,0,0,0,0
1348055,88948836,29400.0,29400.0,60,13.99,683.94,3,C3,9,MORTGAGE,...,0,0,0,0,0,0,0,0,0,0
1348056,89996426,32000.0,32000.0,60,14.49,752.74,3,C4,3,MORTGAGE,...,1,0,0,0,0,0,0,0,0,0
1348057,90006534,16000.0,16000.0,60,12.79,362.34,3,C1,10,RENT,...,0,0,0,1,0,0,0,0,0,0


In [65]:
cols_w_sec_app = [col for col in data.columns if col.startswith('sec_app_')]
data.loc[data.application_type=='Joint App'][cols_w_sec_app]

,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1347918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1348014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1348016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1348022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
cols_w_mths_since = [col for col in data.columns if col.startswith('mths_since_')]
data[cols_w_mths_since]

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq
0,30.0,NaN,30.0,21.0,4.0,69.0,4.0,69.0
1,6.0,NaN,NaN,19.0,2.0,NaN,0.0,6.0
2,NaN,NaN,NaN,19.0,101.0,NaN,10.0,NaN
3,12.0,NaN,NaN,14.0,4.0,12.0,1.0,12.0
4,NaN,NaN,NaN,338.0,36.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1348054,NaN,NaN,NaN,15.0,17.0,NaN,2.0,NaN
1348055,NaN,NaN,NaN,8.0,70.0,NaN,3.0,NaN
1348056,NaN,NaN,NaN,16.0,4.0,NaN,15.0,NaN
1348057,68.0,26.0,68.0,4.0,17.0,NaN,12.0,NaN
